# 🧪 EDA con PySpark - Comportamiento de Usuarios en eCommerce

Este cuaderno utiliza PySpark para realizar un análisis exploratorio de datos (EDA) sobre el archivo `2020-Apr.csv.gz`. Evaluaremos la calidad del dataset, incluyendo valores nulos, duplicados, y formatos erróneos, y documentaremos los pasos necesarios para su limpieza.

## 1.📥 Carga de librerías y creación de sesión de Spark

In [1]:
# Importamos PySpark y funciones necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count

# Creamos una sesión de Spark para el análisis
spark = SparkSession.builder \
    .appName("EDA_Advanced_eCommerce") \
    .getOrCreate()

## 2.📂 Carga del conjunto de datos

In [3]:
# Cargamos el archivo CSV comprimido, indicando encabezado y tipos inferidos
df_full  = spark.read.csv("../data/raw/2020-Apr.csv.gz", header=True, inferSchema=True)

# Visualizamos el esquema del dataset
df_full .printSchema()

# Mostramos las primeras filas como vista rápida
df_full .show(5)

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2020-03-31 19:00:00|      view|   1201465|2232732101407408685|apparel.shoes.sli...| samsung| 230.38|568984877|e2456cef-2d4f-42b...|
|2020-03-31 19:00:01|      view|   1307156|2053013554658804075|electronics.audio...|   a

## 3.📉 Muestreo para el EDA

In [5]:
# Por rendimiento, tomamos una muestra de aproximadamente 1.5 millones de registros
df = df_full.sample(withReplacement=False, fraction=0.023, seed=42)

print(f"Total registros muestreados: {df.count()}")

Total registros muestreados: 1531767


In [6]:
# Eliminar DataFrames temporales para liberar memoria
del df_full

## 4.📏 Verificación de dimensiones y estructura

In [22]:
# Verificar dimensiones
print(f"Número de registros: {df.count()}")
print(f"Número de columnas: {len(df.columns)}")
print("Columnas disponibles:", df.columns)

# Visualización del esquema
df.printSchema()

# Vista rápida de los primeros registros
df.show(5)

Número de registros: 1531767
Número de columnas: 9
Columnas disponibles: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

+-------------------+----------+----------+-------------------+--------------------+------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+-------+---------+--------------------+
|2020-03-31 19:00:11|      view|  

## 5. 💵 Estadísticas Generales de Precio

In [ ]:
# Estadísticas descriptivas de la columna price
df.describe(["price"]).show()

## 6. 🚨 Análisis de calidad de datos

### 📌 A. Conteo de valores nulos por columna

In [ ]:
# Contar valores nulos o NaN por columna
df.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c)
    for c in df.columns
]).show(truncate=False)

### 📌 B. Verificación de duplicados

In [ ]:
# Comparar conteo total con registros únicos
total_registros = df.count()
registros_sin_duplicados = df.dropDuplicates().count()
duplicados = total_registros - registros_sin_duplicados

print(f"🔁 Duplicados detectados: {duplicados}")

### 📌 C. Conteo de valores únicos por columna

In [ ]:
# Recuento de valores únicos por columna
for columna in df.columns:
    print(f"🔍 Valores únicos en '{columna}': {df.select(columna).distinct().count()}")

### 📌 D. Distribución de tipos de eventos